In [0]:
from pyspark.sql.functions import current_timestamp, lit, col

def ingest_bronze_from_metadata(
    metadata_path: str,
    landing_base_path: str,    # raíz de tus archivos raw/landing
    catalog_name: str = None # ej: "main" o "hive_metastore" o None
):
    """
    Lee el CSV de metadatos y, por cada fila, ingesta a una tabla Bronze managed (Delta).
    """

    meta_df = (
        spark.read
            .option("header", "true")
            .option("inferSchema", "true")
            .csv(metadata_path)
    )

    for row in meta_df.collect():
        schema_name = row["schema_name"]      # ej: "bronze"
        table_name  = row["table_name"]       # ej: "ventas"
        file_format = row["file_format"]      # ej: "csv"
        has_header  = row["has_header"]       # True/False
        delimiter   = row["delimiter"]        # ej: ","
        pattern_regex = row["file_patterns"]   # ej: ventas.*\.csv

        header_opt = "true" if bool(has_header) else "false"

        pattern_glob = (
            pattern_regex
            .replace(".*", "*")
            .replace("\\.csv", ".csv")
        )

        df_bronze = (
            spark.read
                .format(file_format)
                .option("header", header_opt)
                .option("delimiter", delimiter)
                .option("inferSchema", "true")
                .option("pathGlobFilter", pattern_glob)   # <--- clave
                .load(landing_base_path)                  # solo lee los que matchean
                .withColumn("ingestion_ts", current_timestamp())
        )

        # Aseguramos el schema (database) y armamos el nombre completo de la tabla managed
        if catalog_name:
            spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog_name}.{schema_name}")
            full_table_name = f"{catalog_name}.{schema_name}.{table_name}"
        else:
            dbutils.notebook.exit(f"Crear el catalogo {catalog_name}")

        # Escribe como tabla Delta managed (sin especificar path)
        (
            df_bronze.write
                .format("delta")
                .mode("append")
                .saveAsTable(full_table_name)
        )

In [0]:
ingest_bronze_from_metadata(
    metadata_path="/Volumes/webinar/default/landing/config/config.csv",
    landing_base_path="/Volumes/webinar/default/landing",
    catalog_name="webinar"   # o "main", o None
)